# 채팅 완료 클라이언트 사용 방법

EasyLLM은 `gpt-3.5-turbo` 및 `gpt-4`를 오픈 소스 모델로 대체하는 추상화 계층으로 사용할 수 있습니다.

클라이언트만 변경하여 OpenAI API에서 자체 애플리케이션을 변경할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 사용하고 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 예제 API 호출을 통해 채팅 형식을 보여줍니다.

### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

In [4]:
# EasyLLM API를 호출하기 위한 EasyLLM Python 라이브러리를 가져옵니다.
import easyllm

### 2. 채팅 API 호출 예제

채팅 API 호출에는 두 가지 필수 입력이 있습니다.
- `model`: 사용하려는 모델의 이름(예: `meta-llama/Llama-2-70b-chat-hf`) 또는 API만 호출하려면 비워 둡니다.
- `messages`: 메시지 개체 목록이며 각 개체에는 두 가지 필수 필드가 있습니다.
    - `role`: 메신저의 역할( `system`, `user` 또는 `assistant`)
    - `content`: 메시지 내용(예: `아름다운 시를 써주세요`)

OpenAI API와 비교하여 `huggingface` 모듈은 프롬프트 및 중지 시퀀스를 사용자 지정하는 데 사용할 수 있는 `prompt_builder` 및 `stop_sequences` 매개변수도 노출합니다. EasyLLM 패키지에는 프롬프트 빌더 유틸리티가 함께 제공됩니다.

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [1]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["HUGGINGFACE_PROMPT"] = "falcon" # vicuna, wizardlm, stablebeluga, open_assistant
# os.environ["HUGGINGFACE_TOKEN"] = "hf_xxx" 

from easyllm.clients import huggingface
from easyllm.prompt_utils.falcon import falcon_stop_sequences

MODEL="tiiuae/falcon-180B-chat"

response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "\n당신은 도움이 되고 정중하며 정직한 조수입니다. 항상 안전하면서 가능한 한 도움이 되도록 답변하십시오. 답변에는 유해하거나 비윤리적이거나 인종 차별적이거나 성 차별적이거나 유독하거나 위험하거나 불법적인 내용이 포함되어서는 안 됩니다. 답변이 사회적으로 편향되지 않고 긍정적인지 확인하십시오.\n\n질문이 이해가 되지 않거나 사실적으로 일관성이 없는 경우 올바르지 않은 내용을 답변하는 대신 이유를 설명하십시오. 질문에 대한 답변을 모르는 경우 거짓 정보를 공유하지 마십시오."},
        {"role": "user", "content": "똑똑."},
        {"role": "assistant", "content": "누구세요?"},
        {"role": "user", "content": "고양이."},
    ],
      temperature=0.9,
      top_p=0.6,
      max_tokens=1024,
      stop=falcon_stop_sequences,
)
response

{'id': 'hf-ceVG8KGm04',
 'object': 'chat.completion',
 'created': 1695106309,
 'model': 'tiiuae/falcon-180B-chat',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "*똑똑* 누구세요? 고양이. 고양이 누구? 고양이가 혀를 물었나?\n사용자:"},
   'finish_reason': 'stop_sequence'}],
 'usage': {'prompt_tokens': 144, 'completion_tokens': 23, 'total_tokens': 167}}

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `chat.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `message`: 모델에서 생성한 메시지 개체( `role` 및 `content` 포함)
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `index`: 선택 항목 목록에서 완성의 인덱스

다음을 사용하여 회신만 추출합니다.

In [2]:
print(response['choices'][0]['message']['content'])

*똑똑* 누구세요? 고양이. 고양이 누구? 고양이가 혀를 물었나?
사용자:


대화 기반이 아닌 작업도 첫 번째 사용자 메시지에 지침을 배치하여 채팅 형식에 맞출 수 있습니다.

예를 들어 모델에게 해적 블랙비어드 스타일로 비동기 프로그래밍을 설명하도록 요청하려면 다음과 같이 대화를 구성할 수 있습니다.

In [3]:
# 시스템 메시지가 있는 예제
response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "당신은 도움이 되는 조수입니다."},
        {"role": "user", "content": "수학 교사 스타일로 비동기 프로그래밍을 설명해주세요."},
    ],
    stop=falcon_stop_sequences,
)

print(response['choices'][0]['message']['content'])



비동기 프로그래밍: 수학적 접근 방식

안녕하세요, 여러분! 오늘 우리는 프로그래밍 세계에서 흥미로운 주제인 비동기 프로그래밍에 대해 논의할 것입니다. 이제 이것이 수학과 무슨 관련이 있는지 궁금할 것입니다. 글쎄요, 수학 연산이 때때로 동기적이거나 비동기적일 수 있는 것처럼 컴퓨터 프로그램도 마찬가지입니다.

먼저 용어를 정의해 보겠습니다. 동기 프로세스는 예측 가능한 순서로 차례로 발생하는 프로세스입니다. 예를 들어 두 숫자를 더한 다음 그 결과에 다른 숫자를 곱하는 경우 이러한 작업은 일반적으로 동기적으로 발생합니다. 즉, 덧셈이 먼저 발생하고 곱셈이 그 뒤를 따릅니다.

반면에 비동기 프로세스는 반드시 그러한 엄격한 순서를 따르지는 않습니다. 기하학의 평행선과 더 비슷합니다. 즉, 서로 완료되기를 기다리지 않고 독립적으로 나란히 실행될 수 있습니다. 프로그래밍에서 이는 한 작업이 다른 작업의 시작을 차단하지 않고 동시에 여러 작업을 수행할 수 있음을 의미합니다.

그렇다면 이것이 왜 유용할까요? 글쎄요, 여러 계산이 필요한 복잡한 수학 문제를 풀고 있다고 상상해 보십시오. 이러한 계산을 동기적으로 수행한다면 각 계산이 완료될 때까지 기다렸다가 다음 계산을 시작해야 합니다. 특히 계산이 사용자 입력이나 네트워크 대기 시간과 같은 외부 요인에 의존하는 경우 시간이 꽤 걸릴 수 있습니다.

그러나 비동기 프로그래밍을 사용하면 여러 계산을 동시에 수행할 수 있습니다. 즉, 한 계산이 사용자 입력을 기다리는 동안 다른 계산은 다른 소스에서 데이터를 계속 처리할 수 있습니다. 결과적으로 전체 계산 시간이 줄어들어 프로그램이 더 효율적이고 응답성이 향상됩니다.

물론 복잡한 수학 퍼즐을 푸는 것과 마찬가지로 비동기 프로그래밍에는 어려움이 따릅니다. 한 가지 주요 문제는 모든 비동기 작업이 도중에 오류가 발생하더라도 성공적으로 완료되도록 하는 것입니다. 이를 위해서는 복잡한 방정식을 푸는 방식과 유사하게 신중한 계획과 오류 처리가 필요합니다.

결론적으로 비동기 프